<a href="https://colab.research.google.com/github/leo02311/Rink/blob/master/lineBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask
!pip install line-bot-sdk
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257
!pip install pyngrok

     |████████████████████████████████| 87 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 29.3 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 56.5 MB/s 
     |████████████████████████████████| 271 kB 60.1 MB/s 
     |████████████████████████████████| 83 kB 1.3 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4
     |████████████████████████████████| 745 kB 1.5 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=7f29fb243aa79181eefbaeb9e649b28849786b8a0a63bd62fdb8d1060d26ad3c
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
     |████████████████████████████████| 804 kB 5.0 MB/s 
  Created wheel for uwsgi: filename=uWSGI-2.0.20-cp37-cp37m-linux_x86_64.whl size=597694 sha256=3c48479d541a79

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok
!/ngrok authtoken 291N4oJwYfwNMWhzCMZTJ4MLOBI_3XHb3thH5BEjMF1G62ezV

Mounted at /content/drive
/drive/ngrok-ssh
--2022-05-12 02:54:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  36.1MB/s    in 0.4s    

2022-05-12 02:54:41 (36.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
from flask import Flask, request, abort    #flask是一個python微型後端框架。

from linebot import (
    LineBotApi, WebhookHandler
)    #讀入以使用linebot的相關模組，模組的用途通常看命名就猜的到。
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir('/content/drive/MyDrive')

/bin/bash: google-drive-ocamlfuse: command not found


In [5]:
!uwsgi app.ini

[uWSGI] getting INI configuration from app.ini
*** Starting uWSGI 2.0.20 (64bit) on [Thu May 12 03:00:51 2022] ***
compiled with version: 7.5.0 on 12 May 2022 02:53:47
os: Linux-5.4.188+ #1 SMP Sun Apr 24 10:03:06 PDT 2022
nodename: 3d625eefa3a7
machine: x86_64
clock source: unix
pcre jit disabled
detected number of CPU cores: 2
current working directory: /content/drive/MyDrive
detected binary path: /usr/local/bin/uwsgi
uWSGI running as root, you can use --uid/--gid/--chroot options
*** WARNING: you are running uWSGI as root !!! (use the --uid flag) *** 
your memory page size is 4096 bytes
detected max file descriptor number: 1048576
lock engine: pthread robust mutexes
thunder lock: disabled (you can enable it with --thunder-lock)
uWSGI http bound on :5000 fd 4
uwsgi socket 0 bound to TCP address 127.0.0.1:44583 (port auto-assigned) fd 3
uWSGI running as root, you can use --uid/--gid/--chroot options
*** WARNING: you are running uWSGI as root !!! (use the --uid flag) *** 
Python versio

In [6]:
import os
import threading

os.environ["FLASK_ENV"] = "development"

from flask_ngrok import run_with_ngrok

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

from pyngrok import ngrok

import random

import configparser 

config = configparser.ConfigParser()
config.read('config.ini')


app = Flask(__name__)
"""
# Open a HTTP tunnel on the default port 80
# <NgrokTunnel: "http://<public_sub>.ngrok.io" -> "http://localhost:80">
public_url = ngrok.connect(port="5000", proto="http", bind_tls=True)
print(public_url)
# Open a SSH tunnel
# <NgrokTunnel: "tcp://0.tcp.ngrok.io:12345" -> "localhost:22">
ssh_tunnel = ngrok.connect(22, "tcp")
"""
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port, bind_tls=True).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL



run_with_ngrok(app)   #starts ngrok when the app is run

line_bot_api = LineBotApi(config.get('line-bot', 'channel_access_token')) #已存於config.ini
handler = WebhookHandler(config.get('line-bot', 'channel_secret'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.source.user_id =='Ucab9e8a853d134c56230256c276b4a35': #填入line在verify時回傳的user_id
        return 'OK'
    else:
      if event.message.text == '鼠倫一個':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="Hi")
        )

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * ngrok tunnel "https://bbb3-34-148-242-181.ngrok.io" -> "http://127.0.0.1:5000"
 * Running on http://bbb3-34-148-242-181.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


https://fwinrgon.herokuapp.com/callback

In [ ]:
from PIL import Image
from numpy import average, linalg, dot
 
 
def get_thumbnail(image, size=(1200, 750), greyscale=False):
    image = image.resize(size, Image.ANTIALIAS)
    if greyscale:
        image = image.convert('L')
    return image
 
 
def image_similarity_vectors_via_numpy(image1, image2):
 
    image1 = get_thumbnail(image1)
    image2 = get_thumbnail(image2)
    images = [image1, image2]
    vectors = []
    norms = []
    for image in images:
        vector = []
        for pixel_tuple in image.getdata():
            vector.append(average(pixel_tuple))
        vectors.append(vector)
        norms.append(linalg.norm(vector, 2))
    a, b = vectors
    a_norm, b_norm = norms
    res = dot(a / a_norm, b / b_norm)
    return res
 
 
image1 = Image.open('1.jpg')
image2 = Image.open('2.jpg')
cosin = image_similarity_vectors_via_numpy(image1, image2)
 
print(cosin)